# Feed Forward Regression


### Importing packages


In [10]:
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets

### Loading data and extracting X and y for specific bidding area


In [11]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

bidding_area = 3

X_train = pd.concat(
    [
        train_data["timestamp"],
        train_data[f"NO{bidding_area}_temperature"].rename("temperature"),
        train_data["time_of_day"],
        train_data["day_of_week"],
        train_data["day_of_year"],
        train_data[f"lagged_mean_NO{bidding_area}_24h"].rename("lagged_mean_24h"),
    ],
    axis="columns",
)

y_train = pd.concat(
    [
        train_data["timestamp"],
        train_data[f"NO{bidding_area}_consumption"].rename("consumption"),
    ],
    axis="columns",
)

X_train.set_index("timestamp", inplace=True)
y_train.set_index("timestamp", inplace=True)

X_train

,temperature,time_of_day,day_of_week,day_of_year,lagged_mean_24h
timestamp,,,,,
2017-05-01 00:00:00+00:00,0.002990,0,0,121,NaN
2017-05-01 01:00:00+00:00,0.002174,1,0,121,NaN
2017-05-01 02:00:00+00:00,0.002011,2,0,121,NaN
2017-05-01 03:00:00+00:00,0.001740,3,0,121,NaN
2017-05-01 04:00:00+00:00,0.001468,4,0,121,NaN
...,...,...,...,...,...
2021-05-01 09:00:00+00:00,0.002011,9,5,121,0.005567
2021-05-01 10:00:00+00:00,0.002174,10,5,121,0.005553
2021-05-01 11:00:00+00:00,0.002718,11,5,121,0.005539


### Creating model


In [12]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")


# Define model
class FeedForward(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


model = FeedForward().to(device)
model

Using mps device


FeedForward(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

### Loss function and optimizer


In [13]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

### Training


In [15]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


X_tensor = torch.tensor(X_train.values, dtype=torch.float32)

AttributeError: 'DataFrame' object has no attribute 'to'

### Testing


In [35]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(
        f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n"
    )

### Run epochs


In [37]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.303525  [   64/60000]
loss: 2.288634  [ 6464/60000]
loss: 2.267837  [12864/60000]
loss: 2.265391  [19264/60000]
loss: 2.247357  [25664/60000]
loss: 2.225983  [32064/60000]
loss: 2.237647  [38464/60000]
loss: 2.201322  [44864/60000]
loss: 2.211879  [51264/60000]
loss: 2.175788  [57664/60000]
Test Error: 
 Accuracy: 43.6%, Avg loss: 2.168451 

Epoch 2
-------------------------------
loss: 2.182071  [   64/60000]
loss: 2.169169  [ 6464/60000]
loss: 2.108543  [12864/60000]
loss: 2.129660  [19264/60000]
loss: 2.080117  [25664/60000]
loss: 2.028171  [32064/60000]
loss: 2.064976  [38464/60000]
loss: 1.980804  [44864/60000]
loss: 2.000977  [51264/60000]
loss: 1.929625  [57664/60000]
Test Error: 
 Accuracy: 54.2%, Avg loss: 1.918648 

Epoch 3
-------------------------------
loss: 1.951860  [   64/60000]
loss: 1.921106  [ 6464/60000]
loss: 1.798057  [12864/60000]
loss: 1.846052  [19264/60000]
loss: 1.733774  [25664/60000]
loss: 1.685287  [32064/600